In [1]:
import pandas as pd
import glob
from io import StringIO
import utils

ALL_SUBJECT_IDS = utils.get_all_subject_ids()

INPUTS_DIR = "outputs/20231209/clusters/"
OUTPUT_DIR = "outputs/20231209/clusters/"
OUTPUT_EXCEL_FILE = f"{OUTPUT_DIR}/clusters_structured.xlsx"


def read_dataframe(content):
    """Try to read the data with tab and comma delimiters. Return the dataframe."""
    try:
        df = pd.read_csv(StringIO(content), sep="\t")
        if df.shape[1] < 2:
            df = pd.read_csv(StringIO(content), sep=",", skipinitialspace=True)
    except Exception as e:
        df = pd.DataFrame()
    return df


data = {}
error_files = []
response_files = sorted(glob.glob(f"{INPUTS_DIR}/A_*.txt"))
print(f"Processing {len(response_files)} response files...")
for file in response_files:
    try:
        # Read the two tables in the file
        # The first table is from the first row to the empty row
        # The second table starts after the empty row
        with open(file, "r") as f:
            # This fails because GPT inserts random newlines
            content = f.read()
            tables = content.strip().split(
                "\n\n"
            )  # Split by two newlines to get the two tables
            table1 = tables[0].strip()
            table2 = tables[1].strip()

            # Remove empty lines before parsing
            # lines = f.readlines()
            # non_empty_lines = [line for line in lines if line.strip()]
            # content = ''.join(non_empty_lines)
            # parts = content.split("subject_id")
            # # The first table is the data before the term "subject_id"
            # table1 = parts[0].strip()
            # # The second table starts with "subject_id" followed by the rest of the content after the split
            # table2 = "subject_id" + parts[1].strip()

            # print(file, len(tables), len(tables[0]), len(tables[1]))
            # print(tables[0][:80])
            # print(tables[0][-80:])
            # print(tables[1][:200])
            # print(tables[1][-80:])

        # Convert into dataframes
        cluster_metadata_df = read_dataframe(table1)
        clustering_df = read_dataframe(table2)
        file_number = int(file.split("_")[-1].split(".")[0])
        data[file_number] = (cluster_metadata_df, clustering_df)

        # Check for errors
        errors = ["|"]
        if cluster_metadata_df.shape[1] != 3:
            errors.append(f"Metadata: {cluster_metadata_df.shape} is not (n, 3)")
        if clustering_df.shape[0] != 93 or clustering_df.shape[1] not in [2, 3]:
            errors.append(f"Clusters: {clustering_df.shape} is not (93, n)")
        missing_subject_ids = set(ALL_SUBJECT_IDS) - set(
            clustering_df["subject_id"].to_list()
        )
        extra_subject_ids = set(clustering_df["subject_id"].to_list()) - set(
            ALL_SUBJECT_IDS
        )
        if missing_subject_ids:
            errors.append(f"Missing subject_ids: {missing_subject_ids}")
        if extra_subject_ids:
            errors.append(f"Extra subject_ids: {extra_subject_ids}")
        print(file, " ".join(errors))
        if errors != ["|"]:
            error_files.append(file)

    except Exception as e:
        error_files.append(file)
        print(f"Error in {file}: {e}")

Processing 42 response files...
outputs/20231209/clusters/A_0.txt |
outputs/20231209/clusters/A_1.txt |
outputs/20231209/clusters/A_10.txt |
outputs/20231209/clusters/A_11.txt |
outputs/20231209/clusters/A_12.txt |
outputs/20231209/clusters/A_13.txt |
outputs/20231209/clusters/A_14.txt |
outputs/20231209/clusters/A_15.txt |
outputs/20231209/clusters/A_16.txt |
outputs/20231209/clusters/A_17.txt |
outputs/20231209/clusters/A_18.txt |
outputs/20231209/clusters/A_19.txt |
outputs/20231209/clusters/A_2.txt |
outputs/20231209/clusters/A_20.txt |
outputs/20231209/clusters/A_21.txt |
outputs/20231209/clusters/A_22.txt |
outputs/20231209/clusters/A_23.txt |
outputs/20231209/clusters/A_24.txt |
outputs/20231209/clusters/A_25.txt |
outputs/20231209/clusters/A_26.txt |
outputs/20231209/clusters/A_27.txt |
outputs/20231209/clusters/A_28.txt |
outputs/20231209/clusters/A_29.txt |
outputs/20231209/clusters/A_3.txt |
outputs/20231209/clusters/A_30.txt |
outputs/20231209/clusters/A_31.txt |
outputs/20

In [2]:
error_files

[]

In [3]:
with pd.ExcelWriter(OUTPUT_EXCEL_FILE) as writer:
    for file_number, (metadata_df, clusters_df) in sorted(data.items()):
        file_number = f"{file_number:02d}"
        metadata_df.to_excel(writer, sheet_name=f"Q{file_number}_metadata", index=False)
        clusters_df.to_excel(writer, sheet_name=f"Q{file_number}_clusters", index=False)

        fn_meta = f"{OUTPUT_DIR}/Q{file_number}_metadata.csv"
        metadata_df.to_csv(fn_meta, index=False)
        fn_clusters = f"{OUTPUT_DIR}/Q{file_number}_clusters.csv"
        clusters_df.to_csv(fn_clusters, index=False)
        print(f"Saved {fn_meta} and {fn_clusters}")

print(f"All data saved to {OUTPUT_EXCEL_FILE}")

Saved outputs/20231209/clusters//Q00_metadata.csv and outputs/20231209/clusters//Q00_clusters.csv
Saved outputs/20231209/clusters//Q01_metadata.csv and outputs/20231209/clusters//Q01_clusters.csv
Saved outputs/20231209/clusters//Q02_metadata.csv and outputs/20231209/clusters//Q02_clusters.csv
Saved outputs/20231209/clusters//Q03_metadata.csv and outputs/20231209/clusters//Q03_clusters.csv
Saved outputs/20231209/clusters//Q04_metadata.csv and outputs/20231209/clusters//Q04_clusters.csv
Saved outputs/20231209/clusters//Q05_metadata.csv and outputs/20231209/clusters//Q05_clusters.csv
Saved outputs/20231209/clusters//Q06_metadata.csv and outputs/20231209/clusters//Q06_clusters.csv
Saved outputs/20231209/clusters//Q07_metadata.csv and outputs/20231209/clusters//Q07_clusters.csv
Saved outputs/20231209/clusters//Q08_metadata.csv and outputs/20231209/clusters//Q08_clusters.csv
Saved outputs/20231209/clusters//Q09_metadata.csv and outputs/20231209/clusters//Q09_clusters.csv
Saved outputs/202312

In [4]:
# data
# Create a merged dataframe
# merged_df = clustering_df.copy()
# cluster_ids = merged_df['cluster_ids'].str.split(',', expand=True)
# for i in range(cluster_ids.shape[1]):
#     col_name = f"cluster_name_{i+1}"
#     merged_df[col_name] = cluster_ids[i].map(cluster_metadata_df.set_index('cluster_id')['cluster_name'])

# cmap = cluster_metadata_df.set_index('cluster_id')['cluster_name']

# clustering_df["secondary_cluster_ids"] #.str.replace("C","") #.map(cmap)
# cmap
# clustering_df["secondary_cluster_ids"].str.split(',', expand=True).stack().map(cmap).unstack()

In [5]:
# ALL_SUBJECT_IDS